# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298169659492                   -0.85    5.0
  2   -8.300171189950       -2.70       -1.26    1.0
  3   -8.300447222129       -3.56       -1.90    4.1
  4   -8.300462465243       -4.82       -2.77    1.2
  5   -8.300464333537       -5.73       -3.05   14.2
  6   -8.300464471021       -6.86       -3.21    1.0
  7   -8.300464553882       -7.08       -3.35    6.6
  8   -8.300464600173       -7.33       -3.49    1.1
  9   -8.300464632807       -7.49       -3.69    1.0
 10   -8.300464636612       -8.42       -3.78    1.9
 11   -8.300464642323       -8.24       -4.00    1.0
 12   -8.300464643961       -8.79       -4.55    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63647483551                   -0.71    5.8
  2   -16.64104810965       -2.34       -1.15    1.0
  3   -16.67920636113       -1.42       -1.75    4.1
  4   -16.67927626325       -4.16       -2.36    3.9
  5   -16.67928349774       -5.14       -2.99    4.0
  6   -16.67928591524       -5.62       -3.33    3.5
  7   -16.67928606594       -6.82       -3.45    2.1
  8   -16.67928622382       -6.80       -4.45    2.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32596049713                   -0.56    7.1
  2   -33.33415921511       -2.09       -1.00    1.4
  3   -33.33603176697       -2.73       -1.73    4.9
  4   -33.33685815905       -3.08       -2.34    7.8
  5   -33.33693961382       -4.09       -2.98    7.6
  6   -33.33694389517       -5.37       -3.83    4.1
  7   -33.33694392148       -7.58       -4.24    4.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298781717370                   -0.85    5.2
  2   -8.300300324484       -2.82       -1.59    1.0
  3   -8.300461187888       -3.79       -2.68    5.0
  4   -8.300443187712   +   -4.74       -2.59    7.0
  5   -8.300464450339       -4.67       -3.62    4.1
  6   -8.300464604986       -6.81       -4.01    2.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.28881530852                   -0.56    6.9
  2   -33.05832405140   +   -0.64       -1.15    1.8
  3   +117.4498254997   +    2.18        0.03   16.5
  4   -32.77032319866        2.18       -1.10    8.2
  5   -31.67888362361   +    0.04       -0.93    4.8
  6   -29.87734991830   +    0.26       -0.79    5.4
  7   -33.31618748219        0.54       -1.77    4.4
  8   -33.32983969475       -1.86       -1.92    2.5
  9   -33.33198494640       -2.67       -1.91    2.5
 10   -33.33518213236       -2.50       -2.35    2.0
 11   -33.33689081240       -2.77       -2.76    2.9
 12   -33.33690812743       -4.76       -2.94    3.1
 13   -33.33693218387       -4.62       -3.16    2.2
 14   -33.33693988414       -5.11       -3.61    2.4
 15   -33.33694351843       -5.44       -4.08    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.